In [1]:
from nanome.service import PluginService
from nanome.interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = '3c07503c-4fc5-4a36-ab23-3aed541ee453'

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_channel)
print('Hello')

Hello


In [5]:
comps = plugin_instance.request_complex_list()
comp = comps[0]
print(comp)


Sending request_complex_list Request to Redis Channel 3c07503c-4fc5-4a36-ab23-3aed541ee453
Response received on channel 6db266e0-7499-44f4-a9e8-7a3c863e3934


In [3]:
pip install py2neo

Note: you may need to restart the kernel to use updated packages.


In [11]:
# neo4j setup
from py2neo import Graph, Node, Relationship

neo4j_uri = 'bolt://neo4j:7687/'
username = 'neo4j'
password = 'F8dwclytew'
graph = Graph(neo4j_uri, auth=(username, password))


In [12]:
with open("assets/neo4j_query.txt") as f:
    query_string = f.read()
results = graph.run(query_string)
data = results.data()

In [2]:
from nanome.api.structure import Atom

atom_list = []
for record in data:
    node = record['a']
    print(next(iter(node.values())))
    continue
    new_atom = Atom(name=next(iter(node.values())))
    atom_list.append()

NameError: name 'data' is not defined

In [2]:
from nanome.api.structure import Complex
comp = Complex()
comp.name = 'Test Complex'

plugin.update_structures_deep([comp])

Sending update_structures_deep Request to Redis Channel 931744fa-b75c-4a08-aad1-c3c772d23138
Response received on channel 5fc3f201-13fd-49db-a5d1-9450b349f0b1


()

In [6]:
from nanome.api.shapes import Shape, Sphere
sphere = Sphere()
sphere.radius = 5
sphere = plugin_instance.upload_shapes([sphere])

Sending upload_shapes Request to Redis Channel 3c07503c-4fc5-4a36-ab23-3aed541ee453
Response received on channel b8b20c68-bc18-462a-9efc-14bc3c77821f
(bytearray(b'\x01'),)


ValueError: not enough values to unpack (expected 2, got 1)

In [4]:
sphere

(bytearray(b'\x01'),)

In [5]:
sphere.decode()

AttributeError: 'tuple' object has no attribute 'decode'